In [7]:
!pip install fastapi uvicorn nest-asyncio pyngrok kagglehub

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import joblib
import kagglehub
import os
from sklearn.metrics import accuracy_score, classification_report

from fastapi import FastAPI
from pydantic import BaseModel, Field, confloat

from pyngrok import ngrok
ngrok.set_auth_token("2zS1ix1LExFP3PbwbOb4zUtovvE_7TmYVGJoGxEp5xCUsD9rA")

import nest_asyncio
import uvicorn


# tải dữ liệu
path = kagglehub.dataset_download("uciml/iris")
csv_path = os.path.join(path, "Iris.csv")
df = pd.read_csv(csv_path)

# chuẩn bị dữ liệu
X = df.drop(columns=["Id", "Species"])
y = df["Species"]

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# train KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
#y_pred_knn = knn.predict(X_test)
#print(f"Độ chính xác (KNN): {accuracy_score(y_test, y_pred_knn):.2f}")
#print("\nClassification Report (KNN):")
#print(classification_report(y_test, y_pred_knn))

# lưu mô hình
joblib.dump(knn, "knn_model.pkl")
#print("Model saved!")

model = joblib.load("knn_model.pkl")
classes = model.classes_  # tên các lớp

class IrisInput(BaseModel):
    SepalLengthCm: confloat(ge=4.3, le=7.9)
    SepalWidthCm: confloat(ge=2.0, le=4.4)
    PetalLengthCm: confloat(ge=1.0, le=6.9)
    PetalWidthCm: confloat(ge=0.1, le=2.5)

app = FastAPI()

@app.get("/")
def home():
    return {"message": "Welcome to Iris Prediction API!", "docs": "/docs"}

@app.post("/predict")
def predict(data: IrisInput):
    df = pd.DataFrame([data.dict()])
    pred = model.predict(df)[0]
    probas = model.predict_proba(df)[0]  # lấy xác suất
    # ghép lớp + xác suất
    proba_dict = dict(zip(classes, [round(float(p), 3) for p in probas]))
    return {
        "prediction": pred,
        "probabilities": proba_dict
    }

# Cho phép chạy event loop lồng nhau (bắt buộc với Colab)
nest_asyncio.apply()

# Kết nối ngrok tới cổng 8000
os.system("killall ngrok || true")
public_url = ngrok.connect(8000).public_url

# Nối /docs để mở Swagger UI
swagger_url = f"{public_url}/docs"

print(f"Server đang chạy tại: {public_url}")
print(f"Truy cập Swagger UI: {swagger_url}")

# Chạy FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

Server đang chạy tại: https://4535-34-90-115-158.ngrok-free.app
Truy cập Swagger UI: https://4535-34-90-115-158.ngrok-free.app/docs


INFO:     Started server process [1074]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2402:800:63b6:af2e:d914:7a6c:6655:20dd:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2402:800:63b6:af2e:d914:7a6c:6655:20dd:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1074]
